In [1]:
import pandas as pd
from haversine import haversine

In [107]:
city_list=["광주", "울산","전주","창원"]
i=2
fire_incident_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+" 화재 정보.csv")
accident_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+"_교통사고_통계.csv")
school_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+" 초중고.csv")
bus_station_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+"시_버스 정류장.csv")
gas_station_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+"_충전소_주유소.csv")
weight_df=pd.DataFrame([[0]*7]*len(gas_station_df), columns=['주유소명', '버정거리총합', '상호개수', '교통사고건수','보호구역개수','화재발생건수','총 가중치'])
weight_df['주유소명']=gas_station_df['상호'][:]
## 추가추가

In [108]:
fire_incident_df.head(3)

,조사서번호,시도,시군구,읍면동,주소,위도,경도
0,'170101093018530,전라북도,전주시덕진구,호성동2가,전라북도 전주시덕진구 호성동2가,35.876378,127.146907
1,'170101100618186,전라북도,전주시덕진구,송천동2가,전라북도 전주시덕진구 송천동2가,35.876378,127.146907
2,'170101135933896,전라북도,전주시완산구,평화동2가,전라북도 전주시완산구 평화동2가,35.876378,127.146907


In [109]:
accident_df.head(3)

,Unnamed: 0,시군구,도로명,주소,사고건수,사망자수,중상자수,경상자수,부상신고자수,위도,경도
0,0,전주시,백제대로,전주시 백제대로,63,1,23,94,0,35.842997,127.139671
1,1,전주시,기린대로,전주시 기린대로,15,1,3,14,0,35.847457,127.112191
2,2,전주시,기린대로,전주시 기린대로,87,1,23,102,1,35.847457,127.112191


In [110]:
bus_station_df.head(3)

,설치 지점명,지역(구),지역(동),세부주소,행정동,위도,경도
0,우아롯데아파트,덕진구,우아동,우아동 2동 887,우아2동,35.832510,127.167824
1,여의동우체국전북은행,덕진구,여의동,여의동2가 696,여의동,35.871416,127.076584
2,기린대로경제통상진흥원,덕진구,팔복동,팔복동2가 550,팔복동,35.859761,127.092527


In [111]:
gas_station_df.head(3)

,지역,상호,타입,주소,위도,경도
0,전라북도,직영충전소,충전소,전북 전주시 덕진구 혁신로 360번지,35.864542,127.069401
1,전라북도,(유)삼영에너지,충전소,전북 전주시 완산구 쑥고개로 11 (용복동),35.790810,127.068859
2,전라북도,(유)푸른기업충전소,충전소,전북 전주시 완산구 모악로 4483 (석구동),35.786460,127.128841


## 가산

가능 : 타 주유소, 교통량(교통 사고), 버스 정류장

working on : 차고지 위치

In [112]:
# 거리 총합 리스트
def 거리총합리스트(data1, data2, name_col):
    # distance_list=[]

    for i in range(len(data1)):
        total=0
        for j in range(len(data2)):
            total+=haversine((data1.loc[i]['위도'],data1.loc[i]['경도']),(data2.loc[j]['위도'],data2.loc[j]['경도']), unit = 'km')
        # distance_list.append((data1.loc[i][name_col],total))
        weight_df['버정거리총합'][i]=total
    

In [113]:
# p 반경 이내 개수 리스트(타 주유소)
def 반경이내_개수_리스트(p, data1, data2, name_col):
    count_list=[]
    for i in range(len(data1)):
        count=0
        for j in range(len(data2)):
            dist=haversine((data1.loc[i]['위도'],data1.loc[i]['경도']),(data2.loc[j]['위도'],data2.loc[j]['경도']), unit = 'km')
            if dist<=p:
                # 교통사고 데이터 사고 건수로 count!
                if ('사고건수' in data2.columns):
                    count+=data2.loc[j,'사고건수']
                else:
                    count+=1
        #count_list.append((data1.loc[i][name_col],count-1)) 상호명 매핑 화긴~
        if (count == 0):
            count_list.append(count)
        else:
            count_list.append(count-1)# 자기 자신 제외

    return count_list
   

In [114]:
""" 이건 km
거리총합리스트(gas_station_df, bus_station_df, '상호')
weight_df['버정거리총합']=max(weight_df['버정거리총합'])-weight_df['버정거리총합']
weight_df
""" # 이건 개수
버정거리총합 = 반경이내_개수_리스트(2, gas_station_df, bus_station_df,'상호')
weight_df['버정거리총합']=버정거리총합
weight_df

,주유소명,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,총 가중치
0,직영충전소,27,0,0,0,0,0
1,(유)삼영에너지,2,0,0,0,0,0
2,(유)푸른기업충전소,59,0,0,0,0,0
3,㈜진성에너지 색장충전소,3,0,0,0,0,0
4,전주유성LPG충전소,4,0,0,0,0,0
...,...,...,...,...,...,...,...
193,(주)안국에너지 호반주유소,79,0,0,0,0,0
194,동아주유소,76,0,0,0,0,0
195,중앙주유소,14,0,0,0,0,0
196,덕산주유소,3,0,0,0,0,0


In [115]:
상호개수 = 반경이내_개수_리스트(2, gas_station_df, gas_station_df,'상호')
weight_df['상호개수']=상호개수
weight_df

,주유소명,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,총 가중치
0,직영충전소,27,29,0,0,0,0
1,(유)삼영에너지,2,3,0,0,0,0
2,(유)푸른기업충전소,59,19,0,0,0,0
3,㈜진성에너지 색장충전소,3,7,0,0,0,0
4,전주유성LPG충전소,4,9,0,0,0,0
...,...,...,...,...,...,...,...
193,(주)안국에너지 호반주유소,79,20,0,0,0,0
194,동아주유소,76,16,0,0,0,0
195,중앙주유소,14,13,0,0,0,0
196,덕산주유소,3,7,0,0,0,0


In [116]:
######이거 수정 : 도로당 회수 있음, 거리 기준으로 함수 변경-> 거리가 작을수록, 회수가 클 수록 좋은,,데,,,
교통사고건수=반경이내_개수_리스트(2, gas_station_df, accident_df,'상호')
weight_df['교통사고건수']=교통사고건수
weight_df

,주유소명,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,총 가중치
0,직영충전소,27,29,62,0,0,0
1,(유)삼영에너지,2,3,2,0,0,0
2,(유)푸른기업충전소,59,19,185,0,0,0
3,㈜진성에너지 색장충전소,3,7,12,0,0,0
4,전주유성LPG충전소,4,9,12,0,0,0
...,...,...,...,...,...,...,...
193,(주)안국에너지 호반주유소,79,20,423,0,0,0
194,동아주유소,76,16,228,0,0,0
195,중앙주유소,14,13,26,0,0,0
196,덕산주유소,3,7,12,0,0,0


## 감점

가능 : 어린이집, 유초중고(2km)

working on : 화재

In [117]:
fire_incident_df.head(3)

,조사서번호,시도,시군구,읍면동,주소,위도,경도
0,'170101093018530,전라북도,전주시덕진구,호성동2가,전라북도 전주시덕진구 호성동2가,35.876378,127.146907
1,'170101100618186,전라북도,전주시덕진구,송천동2가,전라북도 전주시덕진구 송천동2가,35.876378,127.146907
2,'170101135933896,전라북도,전주시완산구,평화동2가,전라북도 전주시완산구 평화동2가,35.876378,127.146907


In [118]:
gas_station_df.head(3)

,지역,상호,타입,주소,위도,경도
0,전라북도,직영충전소,충전소,전북 전주시 덕진구 혁신로 360번지,35.864542,127.069401
1,전라북도,(유)삼영에너지,충전소,전북 전주시 완산구 쑥고개로 11 (용복동),35.790810,127.068859
2,전라북도,(유)푸른기업충전소,충전소,전북 전주시 완산구 모악로 4483 (석구동),35.786460,127.128841


In [119]:
school_df.head(3)

,시도,행정구,교육\n(지원)청,학교급,고등학교\n유형,KEDI 학제,학교명,우편번호,주소,위도,경도
0,전북,전주시,전북,고등학교,일반고,일반고등학교,전북대학교사범대학부설고등학교,54896,전라북도 전주시 덕진구 백제대로 567(금암동. 전북대학교),35.846732,127.129367
1,전북,전주시,전북,고등학교,일반고,일반고등학교,전라고등학교,54836,전라북도 전주시 덕진구 솔내7길 25(송천동1가. 전라고등학교),35.859966,127.124891
2,전북,전주시,전북,고등학교,일반고,일반고등학교,전주고등학교,55007,전라북도 전주시 완산구 권삼득로 2(중노송동. 전주고등학교),35.825388,127.151833


In [120]:
kinder_df.head(3)

,Unnamed: 0,어린이집명,주소,위도,경도
0,0,e편한세상어린이집,"전라북도 전주시 완산구 여울로 161 관리동(서신동, e편한세상)",35.839530,127.106910
1,1,NPS국민연금어린이집,전라북도 전주시 덕진구 기지로 180,35.837661,127.071757
2,2,SK뷰 생글어린이집,"전라북도 전주시 완산구 태평2길 22 관리동(태평동, 태평SKVIEW)",35.824693,127.139501


In [121]:
#유치원,,
#보호구역개수=반경이내_개수_리스트(2, gas_station_df, kinder_df, '상호')
#weight_df['보호구역개수']=보호구역개수

In [122]:
보호구역개수=반경이내_개수_리스트(2, gas_station_df, school_df, '상호')
weight_df['보호구역개수']=보호구역개수
weight_df

,주유소명,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,총 가중치
0,직영충전소,27,29,62,11,0,0
1,(유)삼영에너지,2,3,2,4,0,0
2,(유)푸른기업충전소,59,19,185,37,0,0
3,㈜진성에너지 색장충전소,3,7,12,0,0,0
4,전주유성LPG충전소,4,9,12,1,0,0
...,...,...,...,...,...,...,...
193,(주)안국에너지 호반주유소,79,20,423,34,0,0
194,동아주유소,76,16,228,47,0,0
195,중앙주유소,14,13,26,3,0,0
196,덕산주유소,3,7,12,0,0,0


In [123]:
화재발생건수=반경이내_개수_리스트(2, gas_station_df, fire_incident_df, '상호')
weight_df['화재발생건수']=화재발생건수
weight_df

,주유소명,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,총 가중치
0,직영충전소,27,29,62,11,8,0
1,(유)삼영에너지,2,3,2,4,1,0
2,(유)푸른기업충전소,59,19,185,37,34,0
3,㈜진성에너지 색장충전소,3,7,12,0,3,0
4,전주유성LPG충전소,4,9,12,1,3,0
...,...,...,...,...,...,...,...
193,(주)안국에너지 호반주유소,79,20,423,34,43,0
194,동아주유소,76,16,228,47,37,0
195,중앙주유소,14,13,26,3,16,0
196,덕산주유소,3,7,12,0,3,0


## 스케일링

In [124]:
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler()
print(minMaxScaler.fit(weight_df.iloc[:,[1,2,3,4,5]]))
weight_df_stdScaled = minMaxScaler.transform(weight_df.iloc[:,[1,2,3,4,5]])
weight_df_stdScaled=pd.DataFrame(weight_df_stdScaled, columns=["버정거리총합","상호개수","교통사고건수","보호구역개수","화재발생건수"])
weight_df_stdScaled["주유소명"]=weight_df["주유소명"][:]
weight_df_stdScaled

MinMaxScaler()


,버정거리총합,상호개수,교통사고건수,보호구역개수,화재발생건수,주유소명
0,0.190141,0.852941,0.093093,0.177419,0.093023,직영충전소
1,0.014085,0.088235,0.003003,0.064516,0.011628,(유)삼영에너지
2,0.415493,0.558824,0.277778,0.596774,0.395349,(유)푸른기업충전소
3,0.021127,0.205882,0.018018,0.000000,0.034884,㈜진성에너지 색장충전소
4,0.028169,0.264706,0.018018,0.016129,0.034884,전주유성LPG충전소
...,...,...,...,...,...,...
193,0.556338,0.588235,0.635135,0.548387,0.500000,(주)안국에너지 호반주유소
194,0.535211,0.470588,0.342342,0.758065,0.430233,동아주유소
195,0.098592,0.382353,0.039039,0.048387,0.186047,중앙주유소
196,0.021127,0.205882,0.018018,0.000000,0.034884,덕산주유소


In [125]:
weight_df_stdScaled.to_csv(city_list[i]+" 스케일링완료.csv")

# ########## 여기까지1 - 스케일링 완, 가중치 미완(lr용) ############

## 가중치 매핑 함수
새로운 데이터 프레임 만들어서 '상호'와 가중치1,2,3,4,5 넣고 총합 마지막에 하고 sort해서 [:20] 출력

### 가산
주유소_count_list=반경이내_개수_리스트(2, gas_station_df, gas_station_df,'상호')

distance_list=거리총합리스트(gas_station_df, bus_station_df, '상호')

교통사고_count_list=반경이내_개수_리스트(2, gas_station_df, accident_df,'상호')
### 감점
유치원_count_list=반경이내_개수_리스트(2, gas_station_df, kinder_df, '상호')

유초중고_count_list=반경이내_개수_리스트(2, gas_station_df, school_df, '상호')

화재_count_list=반경이내_개수_리스트(2, gas_station_df, fire_incident_df, '상호')

In [ ]:
# **주유소 순서 헤치지 말 것**
#columns=['버정거리총합', '상호개수', '교통사고건수','보호구역개수','화재발생건수',주유소명'])
def 가중치추가(df, ratio_list):
    for i in range(0,len(df.loc[0])-2):
        df['총 가중치']+=df.iloc[:,i]*ratio_list[i]# weight_df는 주유소명부터, ratio_list는 버정거리총합부터 시작하므로
    


In [ ]:
weight_df_stdScaled["총 가중치"]=0 # 혹시 모를 사고에 대비
ratio_list=[0.001,0.1,0.1,-0.1,-0.1] # 뒤에 두개 음수 필수
가중치추가(weight_df_stdScaled, ratio_list)
weight_df_stdScaled.head(5)

In [ ]:
weight_df_stdScaled_sorted=weight_df_stdScaled.sort_values(by=["총 가중치", "주유소명"], ascending=[False, False]) 
weight_df_stdScaled_sorted

In [ ]:
weight_df_sorted.to_csv(city_list[2]+" 가중치.csv")

# ########## 여기까지2 - 스케일링, 가중치 모두 완 (f1용)

## 정확도 나타내기
가중치가 높은 상위 20개 중 수소 충전소가 있는 개수
- 전체 충전소 개수 중 맞춘 개수?

In [ ]:
# 예시예시 10개!! 실제로는 불러와야겠죠,,,
guess_df=pd.DataFrame([[0]*2]*10, columns=['주유소명','어쩌구'])
# 찐 걍 예시

guess_df['주유소명']=gas_station_df['상호'][:10]
guess_df

In [ ]:
# 예시 친구들을 전체 주유소에 대입
labels_df=pd.DataFrame([[0]*2]*len(weight_df_sorted), columns=['주유소명','label'])
labels_df['주유소명']=gas_station_df['상호']
for i in range(len(guess_df)):
    if labels_df['주유소명'][i]==guess_df['주유소명'][i]:
        labels_df['label'][i]+=1

In [ ]:
# weight_df_sorted 상위 10개 label에 1 넣음
eval_df=pd.DataFrame([[0]*2]*len(weight_df_sorted), columns=['주유소명','label'])
eval_df['주유소명']=weight_df_sorted['주유소명'][:]
eval_df['label'][:len(guess_df)]=1#일단 그냥 추측 개수 같게 했다,,

In [ ]:
# 전체 충전소 개수 중 guess의 개수 안에 있는 충전소 개수
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
eval_df['label']=0
# eval_df.columns=['주유소명','label']
for i in range(len(guess_df)):
    if guess_df['주유소명'][i]==eval_df['주유소명'][i]:
        eval_df['label'][i]+=1

eval_df.sort_values(by=["주유소명"], ascending=[False]) 
guess_df.sort_values(by=["주유소명"], ascending=[False]) 
# 원래 친구
labels=labels_df['label']
# 추정한 친구
guesses=eval_df['label']
guesses

In [ ]:
print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses))
print(precision_score(labels, guesses))
print(f1_score(labels, guesses))

## 동 매핑

동이랑 통행속도 매핑

gas_station_df['주소']-도로 이름 나오니까 그냥 사용

travel_speed['도로명'], ['속도']

In [ ]:
# 그냥 대충 만든 거
travel_speed=pd.DataFrame([['혁신로', 20],['기지로', 30],['쑥고개로', 10],['권삼득로', 33]], columns=['도로명','속도'])
travel_speed

In [ ]:
# 가중치 데이터 프레임
weight_df=pd.DataFrame([[0]*5]*len(gas_station_df), columns=['주유소_이름', '주소','통행속도_w', 'a', 'b'])
weight_df['주유소_이름']=gas_station_df['상호'][:]
weight_df['주소']=gas_station_df['주소'][:]
weight_df

In [ ]:
for i in range(len(travel_speed)):
    for j in range(len(gas_station_df)):
        if travel_speed.loc[i]['도로명'] in gas_station_df.loc[j]['주소']:
            # gas_station_df.loc[j]['상호']
            # 하는 중
            # weight_df.iloc['주유소_이름']['통행속도_w']+=1
weight_df